
**CODIGO FUNCIONAL**





In [ ]:
# Instalar dependencias necesarias
!pip install kagglehub fastapi uvicorn nest-asyncio pyngrok joblib

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import joblib
import os
import kagglehub

# Descargar dataset
path = kagglehub.dataset_download("yashdevladdha/uber-ride-analytics-dashboard")
df = pd.read_csv(f"{path}/ncr_ride_bookings.csv")

# 🧹 Preprocesamiento
df = df[df["Booking Status"] == "Completed"].copy()
df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str))
df["Hour"] = df["DateTime"].dt.hour
df["DayOfWeek"] = df["DateTime"].dt.dayofweek
df["IsWeekend"] = df["DayOfWeek"].isin([5, 6]).astype(int)
df["IsPeakHour"] = df["Hour"].isin([7, 8, 9, 17, 18, 19]).astype(int)

# Codificación One-Hot con manejo de categorías desconocidas
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoder.fit(df[["Vehicle Type"]])
vehicle_encoded = encoder.transform(df[["Vehicle Type"]])
vehicle_df = pd.DataFrame(vehicle_encoded, columns=encoder.get_feature_names_out(["Vehicle Type"]))

#  Combinar con el dataframe original
df = pd.concat([df.reset_index(drop=True), vehicle_df], axis=1)

# Selección de variables
features = ["Hour", "DayOfWeek", "IsWeekend", "IsPeakHour", "Ride Distance"] + list(vehicle_df.columns)
X = df[features]
y = df["Booking Value"]

# Entrenamiento del modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Evaluación
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse:.2f}")

# Guardar modelo y encoder
joblib.dump(model, "gradient_boosting_model.pkl")
joblib.dump(encoder, "vehicle_encoder.pkl")


Using Colab cache for faster access to the 'uber-ride-analytics-dashboard' dataset.
Error Cuadrático Medio (MSE): 150985.44


['vehicle_encoder.pkl']

In [ ]:
# Importar librerías para la API
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading
import time
import requests

# 🧩 Activar compatibilidad con Colab
nest_asyncio.apply()

# Configurar ngrok
ngrok.set_auth_token("33X41rhmDQpmvew1rCUaavqsuLz_6djyJawZD7azXwYArNmLH")

# Cargar modelo y encoder
model = joblib.load("gradient_boosting_model.pkl")
encoder = joblib.load("vehicle_encoder.pkl")
print(" Modelo y codificador cargados correctamente")

# Crear la API
app = FastAPI()

class RideFeatures(BaseModel):
    hour: int
    day_of_week: int
    is_weekend: int
    is_peak_hour: int
    ride_distance: float
    vehicle_type: str

@app.post("/predict")
def predict_price(features: RideFeatures):
    try:
        vehicle_encoded = encoder.transform([[features.vehicle_type]])
        vehicle_df = pd.DataFrame(vehicle_encoded, columns=encoder.get_feature_names_out(["Vehicle Type"]))
        input_data = pd.DataFrame([{
            "Hour": features.hour,
            "DayOfWeek": features.day_of_week,
            "IsWeekend": features.is_weekend,
            "IsPeakHour": features.is_peak_hour,
            "Ride Distance": features.ride_distance
        }])
        input_data = pd.concat([input_data, vehicle_df], axis=1)
        prediction = model.predict(input_data)[0]
        return {"valor_estimado_del_viaje": round(prediction, 2)}
    except Exception as e:
        return {"error": str(e)}

# Crear túnel ngrok
public_url = ngrok.connect(8000).public_url
print(f" Tu API está disponible en: {public_url}")

#  Lanzar servidor en segundo plano
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if not any(thread.name == "ServerThread" for thread in threading.enumerate()):
    server_thread = threading.Thread(target=run_server, name="ServerThread")
    server_thread.start()

# Esperar a que el servidor esté listo
time.sleep(5)

# Probar la API
url = f"{public_url}/predict"
data = {
    "hour": 23,
    "day_of_week": 6,
    "is_weekend": 1,
    "is_peak_hour": 0,
    "ride_distance": 3.5,
    "vehicle_type": "Mini"
}
response = requests.post(url, json=data)
print("Status:", response.status_code)
print("Respuesta:", response.json())


 Modelo y codificador cargados correctamente
 Tu API está disponible en: https://unprobational-hyenoid-rebecka.ngrok-free.dev
INFO:     34.133.205.139:0 - "POST /predict HTTP/1.1" 200 OK
Status: 200
Respuesta: {'valor_estimado_del_viaje': 682.6}


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
